## 문서 유사도 - 코사인 유사도를 이용하기

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [1]:
# 파일 읽어서 데이터 프레임으로 변환
import numpy as np
import pandas as pd
import glob, os
# glob : 지정한 파일 목록 반환 모듈

In [2]:
# 리뷰의 파일 목록 구하기
# glob와 os.path.join function 을 이용하여 토픽 디렉토리안의 모든 리뷰 파일 가져오기
all_files = glob.glob(
      os.path.join('./data/uci_reviews/OpinosisDataset1.0/topics/'   # 파일 목록 가져올 위치 지정
    , '*.data')    # 가져올 파일의 패턴 지정
)

['./data/uci_reviews/OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\bathroom_bestwestern_hotel_sfo.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\battery-life_amazon_kindle.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\battery-life_ipod_nano_8gb.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\battery-life_netbook_1005ha.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\buttons_amazon_kindle.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\comfort_honda_accord_2008.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\comfort_toyota_camry_2007.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\directions_garmin_nuvi_255W_gps.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\display_garmin_nuvi_255W_gps.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/topics\\eyesight-issues_amazon_kindle.txt.data',
 './data/uci_reviews/OpinosisDataset1.0/top

### 앞절 문서의 군집화에서 했던, 군집화된 문서의 벡터화된 피처 데이터를 이용하여 코사인 유사도를 확인하기

In [6]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 토큰화 함수 작성
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 토큰분리 객체 생성
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize  # 위에서 만든 토큰화 함수
                             , stop_words='english'
                             , ngram_range=(1,2), min_df=0.05, max_df=0.85 )


In [12]:
import numpy as np
import pandas as pd
import glob, os

all_files = glob.glob(
      os.path.join('./data/uci_reviews/OpinosisDataset1.0/topics/'   # 파일 목록 가져올 위치 지정
    , '*.data')    # 가져올 파일의 패턴 지정
)

# for 문 돌리기

# 파일 이름 저장공간 생성
filename_list = [] 

# 데이터 저장 공간 생성
opinion_text = []

# 파일이름을 이용해서, 데이터 프레임 생성
    
# 개별파일들의 파일내용은 DataFrame 로딩후, 다시 스트링으로 변환하여 opinion_text 리스트로 취합
# 파일 이름 전체 얻기
for file_ in all_files:
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    opinion_text.append(df.to_string())
    filename = file_.split('\\')[-1].split('.')[0]
    filename_list.append(filename)

In [13]:
# 파일명 list와 파일내용 list 객체를 하나의 데이터 프레임으로 말아서 문서 저장하기
doc_df = pd.DataFrame({
    'filename' : filename_list
    ,'opinion_text' : opinion_text
})
doc_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [14]:
#opinion_text 컬럼값으로 feature vectorization 수행
# feature_vect : 군집화할 학습데이터가 됨.
feature_vect = tfidf_vect.fit_transform(doc_df['opinion_text'])

C:\Users\TJ\anaconda3\envs\ml01-02\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [15]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
doc_df['cluster_label'] = cluster_label
doc_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
48,updates_garmin_nuvi_255W_gps,...,0
44,speed_windows7,...,0
43,speed_garmin_nuvi_255W_gps,...,0
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
41,size_asus_netbook_1005ha,...,0
36,screen_netbook_1005ha,...,0
35,screen_ipod_nano_8gb,...,0
34,screen_garmin_nuvi_255W_gps,...,0
33,satellite_garmin_nuvi_255W_gps,...,0


In [16]:
doc_df['cluster_label'][:10]

0    0
1    1
2    0
3    0
4    0
5    0
6    2
7    2
8    0
9    0
Name: cluster_label, dtype: int32

###  호텔로 클러스터링 된 문서들 간의 코사인 유사도 확인

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

# cluster_label=1인 데이터는 호텔로 클러스터링된 데이터임. DataFrame에서 해당 Index를 추출
hotel_indexes = doc_df[doc_df['cluster_label']==1].index
print('호텔로 클러스터링 된 문서들의 DataFrame Index:', hotel_indexes)

호텔로 클러스터링 된 문서들의 DataFrame Index: Int64Index([1, 13, 14, 15, 20, 21, 24, 28, 30, 31, 32, 38, 39, 40, 45, 46], dtype='int64')


In [20]:
# 호텔로 클러스터링된 데이터 중 첫번째 문서를 추출하여 파일명 표시.  
comparison_docname = doc_df.iloc[hotel_indexes[0]]['filename']
print('##### 비교 기준 문서명 ',comparison_docname,' 와 타 문서 유사도######')

''' document_df에서 추출한 Index 객체를 feature_vect로 입력하여 호텔 클러스터링된 feature_vect 추출 
이를 이용하여 호텔로 클러스터링된 문서 중 첫번째 문서와 다른 문서간의 코사인 유사도 측정.'''
similarity_pair = cosine_similarity(feature_vect[hotel_indexes[0]] , feature_vect[hotel_indexes])
print(similarity_pair)

##### 비교 기준 문서명  bathroom_bestwestern_hotel_sfo  와 타 문서 유사도######
[[1.         0.0430688  0.05221059 0.06189595 0.05846178 0.06193118
  0.03638665 0.11742762 0.38038865 0.32619948 0.51442299 0.11282857
  0.13989623 0.1386783  0.09518068 0.07049362]]


In [22]:
doc_df[doc_df['cluster_label']==1]

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,1
13,food_holiday_inn_london,...,1
14,food_swissotel_chicago,...,1
15,free_bestwestern_hotel_sfo,...,1
20,location_bestwestern_hotel_sfo,...,1
21,location_holiday_inn_london,...,1
24,parking_bestwestern_hotel_sfo,...,1
28,price_holiday_inn_london,...,1
30,rooms_bestwestern_hotel_sfo,...,1
31,rooms_swissotel_chicago,...,1


In [27]:
# 가장 코사인 유사도가 높은 애들의 파일이름 확인해보기(기준 문서 : bathroom_bestwestern_hotel_sfo)
comparison_docname = doc_df.iloc[hotel_indexes[8:11]]['filename']
print(comparison_docname)

print(similarity_pair[0][8:11])

30    rooms_bestwestern_hotel_sfo
31        rooms_swissotel_chicago
32        room_holiday_inn_london
Name: filename, dtype: object
[0.38038865 0.32619948 0.51442299]
